In [1]:
# !pip install nbconvert

In [2]:
!pip show huggingface-hub

Name: huggingface-hub
Version: 0.20.2
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: tokenizers, transformers


## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [3]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

from urllib.request import urlretrieve

urlretrieve(
    "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt",
    "input.txt",
)

('input.txt', <http.client.HTTPMessage at 0x168cf44be50>)

In [4]:
# read it in to inspect it
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [5]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [6]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


def encode(s):
    return [stoi[c] for c in s]  # encoder: take a string, output a list of integers


def decode(l):
    return "".join(
        [itos[i] for i in l]
    )  # decoder: take a list of integers, output a string


print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch  # we use PyTorch: https://pytorch.org

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(
    data[:1000]
)  # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# Let's now split up the data into train and validation sets
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [24]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?


def get_batch(split: str) -> tuple[torch.Tensor, torch.Tensor]:
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(f"ix is {ix}")
    # ix is tensor([ 76049, 234249, 934904, 560986])
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")

In [14]:
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [15]:
print(xb)  # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # print(f'embds are: {self.token_embedding_table.weight}, shape: {self.token_embedding_table.weight.shape}')
        # print(f'embds few elements: {self.token_embedding_table.weight[:5,:5]}')
        # 65*65

    def forward(self, idx, targets=None):
        # print("\n****fwd pass********")
        # print(f"idx = {idx}, shape = {idx.shape}")

        # try:
        #     print(f"targets = {targets}, targets = {targets.shape}")
        # except:
        #     print("targets is none type")

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)
        # print(f"logits = {logits}, shape = {logits.shape}")

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # print(f"B, T, C = {B, T, C}")
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            # print(f"now shape logits = {logits.shape}, targets = {targets.shape}")
            loss = F.cross_entropy(logits, targets)
            # print(f"CE loss = {loss}")

        # print("**** exiting fwd pass *******")

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        print(f"idx initially in generate function = {idx}, shape = {idx.shape}")

        for _ in range(max_new_tokens):
            print(f"\n\n max_new_tokens in loop = {_}")
            # get the predictions
            logits, loss = self(idx)
            print(f"logits shape= {logits.shape}, loss = {loss}")
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            print(f"logits reshaped = {logits}, shape= {logits.shape}")
            print(f"logits reshaped shape = {logits.shape}")

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            print(f"probs {probs} shape= {probs.shape}")

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            print(f"idx_next = {idx_next}, shape = {idx_next.shape}")

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
            print(f"idx after concat = {idx}, shape = {idx.shape}")

        return idx

In [23]:
print(f"input data xb = {xb}, shape = {xb.shape}")
print(f"target data yb = {yb}, shape = {yb.shape}")
m = BigramLanguageModel(vocab_size)


input data xb = tensor([[26, 32, 21, 27, 10,  0, 32, 39],
        [47, 52, 43,  1, 51, 53, 52, 58],
        [ 1, 39,  1, 45, 43, 52, 58, 50],
        [39, 56, 43, 52, 41, 43,  5,  1],
        [51, 39, 52,  1, 46, 47, 51, 57],
        [45,  1, 47, 52, 58, 53,  1, 24],
        [46, 39, 60, 43,  1, 52, 53,  1],
        [13, 52, 42,  1, 50, 39, 63,  1],
        [33, 30, 37, 10,  0, 21,  1, 61],
        [57, 43, 10,  0, 21,  1, 57, 54],
        [43, 52, 12,  0,  0, 30, 21, 15],
        [60, 43, 56, 61, 46, 43, 50, 51],
        [50,  1, 63, 53, 59, 56,  1, 50],
        [39, 56, 58,  1, 57, 61, 47, 44],
        [ 1, 53, 44,  1, 46, 47, 57,  1],
        [45, 53,  1, 58, 53, 11,  0, 13],
        [43, 43,  1, 57, 58, 47, 50, 50],
        [53, 53, 56, 57,  2,  0,  0, 34],
        [ 1, 57, 46, 43,  1, 41, 53, 51],
        [52, 11,  1, 58, 46, 43, 63,  1],
        [ 6,  1, 57, 58, 43, 43, 50,  5],
        [46,  1, 39,  1, 40, 50, 53, 53],
        [51, 43, 43, 58,  1, 31, 59, 56],
        [41, 56, 3

In [16]:
logits, loss = m(xb, yb)
print(
    f"initial logits and loss ===> \nlogits {logits}, shape = {logits.shape} \n , loss {loss}, shape = {loss.shape}"
)
print("---------------------")
print("---------------------\n\n\n")
print(f"logits.shape: {logits.shape}")
print(f"loss: {loss}")

input data xb = tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]), shape = torch.Size([4, 8])
target data yb = tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]]), shape = torch.Size([4, 8])

****fwd pass********
idx = tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]), shape = torch.Size([4, 8])
targets = tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]]), targets = torch.Size([4, 8])
logits = tensor([[[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572

In [18]:
inputs = torch.zeros((1, 1), dtype=torch.long)
print(f"inputs = {inputs}")

max_new_tokens = 50

print("calling generate function ....")

output = m.generate(idx=inputs, max_new_tokens=max_new_tokens)

print(f"outputs = {output} \n output shape = {output.shape}")
# tensor([[ 0,...]])
print(f"first element is {output[0]}")
# tensor([ 0,...])
print(f"first element in list is {output[0].tolist()}")
# [0, ...]
print(f"decoded output = {decode(output[0].tolist())}")


inputs = tensor([[0]])
calling generate function ....
idx initially in generate function = tensor([[0]]), shape = torch.Size([1, 1])


 max_new_tokens in loop = 0

****fwd pass********
idx = tensor([[0]]), shape = torch.Size([1, 1])
targets is none type
logits = tensor([[[ 0.1808, -0.0700, -0.3596, -0.9152,  0.6258,  0.0255,  0.9545,
           0.0643,  0.3612,  1.1679, -1.3499, -0.5102,  0.2360, -0.2398,
          -0.9211,  1.5433,  1.3488, -0.1396,  0.2858,  0.9651, -2.0371,
           0.4931,  1.4870,  0.5910,  0.1260, -1.5627, -1.1601, -0.3348,
           0.4478, -0.8016,  1.5236,  2.5086, -0.6631, -0.2513,  1.0101,
           0.1215,  0.1584,  1.1340, -1.1539, -0.2984, -0.5075, -0.9239,
           0.5467, -1.4948, -1.2057,  0.5718, -0.5974, -0.6937,  1.6455,
          -0.8030,  1.3514, -0.2759, -1.5108,  2.1048,  2.7630, -1.7465,
           1.4516, -1.5103,  0.8212, -0.2115,  0.7789,  1.5333,  1.6097,
          -0.4032, -0.8345]]], grad_fn=<EmbeddingBackward0>), shape = torch.Size

idx_next = tensor([[8]]), shape = torch.Size([1, 1])
idx after concat = tensor([[ 0, 31, 28, 63, 39, 53,  8]]), shape = torch.Size([1, 7])


 max_new_tokens in loop = 6

****fwd pass********
idx = tensor([[ 0, 31, 28, 63, 39, 53,  8]]), shape = torch.Size([1, 7])
targets is none type
logits = tensor([[[ 0.1808, -0.0700, -0.3596, -0.9152,  0.6258,  0.0255,  0.9545,
           0.0643,  0.3612,  1.1679, -1.3499, -0.5102,  0.2360, -0.2398,
          -0.9211,  1.5433,  1.3488, -0.1396,  0.2858,  0.9651, -2.0371,
           0.4931,  1.4870,  0.5910,  0.1260, -1.5627, -1.1601, -0.3348,
           0.4478, -0.8016,  1.5236,  2.5086, -0.6631, -0.2513,  1.0101,
           0.1215,  0.1584,  1.1340, -1.1539, -0.2984, -0.5075, -0.9239,
           0.5467, -1.4948, -1.2057,  0.5718, -0.5974, -0.6937,  1.6455,
          -0.8030,  1.3514, -0.2759, -1.5108,  2.1048,  2.7630, -1.7465,
           1.4516, -1.5103,  0.8212, -0.2115,  0.7789,  1.5333,  1.6097,
          -0.4032, -0.8345],
         [-1.0699, -

In [ ]:
# new tokens aren't the same everytime because we are sampling from multinomial dist

In [20]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)  # from 1e-3

In [27]:
batch_size = 32  # overwrites batch_size for the generate_data function

for steps in range(1000):  # increase number of steps for good results...
    # sample a (new/random) batch of data
    xb, yb = get_batch("train")

    # print(f'input data xb = {xb}, shape = {xb.shape}')
    # print(f'target data yb = {yb}, shape = {yb.shape}')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 10 == 0:
        print(f"step: {steps} \t loss: {loss}")

print(loss.item())

step: 0 	 loss: 4.790024280548096
step: 10 	 loss: 4.701449394226074
step: 20 	 loss: 4.748788356781006
step: 30 	 loss: 4.7372283935546875
step: 40 	 loss: 4.642716407775879
step: 50 	 loss: 4.668173789978027
step: 60 	 loss: 4.652814865112305
step: 70 	 loss: 4.743861198425293
step: 80 	 loss: 4.760161876678467
step: 90 	 loss: 4.6539812088012695
step: 100 	 loss: 4.622498989105225
step: 110 	 loss: 4.721787929534912
step: 120 	 loss: 4.814096450805664
step: 130 	 loss: 4.591794013977051
step: 140 	 loss: 4.84165620803833
step: 150 	 loss: 4.74301815032959
step: 160 	 loss: 4.694024562835693
step: 170 	 loss: 4.674234867095947
step: 180 	 loss: 4.664593696594238
step: 190 	 loss: 4.704473972320557
step: 200 	 loss: 4.68118143081665
step: 210 	 loss: 4.782592296600342
step: 220 	 loss: 4.711303234100342
step: 230 	 loss: 4.759653091430664
step: 240 	 loss: 4.780046463012695
step: 250 	 loss: 4.696370601654053
step: 260 	 loss: 4.685718059539795
step: 270 	 loss: 4.7214813232421875
ste

In [28]:
print(
    decode(
        m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[
            0
        ].tolist()
    )
)

idx initially in generate function = tensor([[0]]), shape = torch.Size([1, 1])


 max_new_tokens in loop = 0
logits shape= torch.Size([1, 1, 65]), loss = None
logits reshaped = tensor([[ 0.1808, -0.0700, -0.3596, -0.9152,  0.6258,  0.0255,  0.9545,  0.0643,
          0.3612,  1.1679, -1.3499, -0.5102,  0.2360, -0.2398, -0.9211,  1.5433,
          1.3488, -0.1396,  0.2858,  0.9651, -2.0371,  0.4931,  1.4870,  0.5910,
          0.1260, -1.5627, -1.1601, -0.3348,  0.4478, -0.8016,  1.5236,  2.5086,
         -0.6631, -0.2513,  1.0101,  0.1215,  0.1584,  1.1340, -1.1539, -0.2984,
         -0.5075, -0.9239,  0.5467, -1.4948, -1.2057,  0.5718, -0.5974, -0.6937,
          1.6455, -0.8030,  1.3514, -0.2759, -1.5108,  2.1048,  2.7630, -1.7465,
          1.4516, -1.5103,  0.8212, -0.2115,  0.7789,  1.5333,  1.6097, -0.4032,
         -0.8345]], grad_fn=<SliceBackward0>), shape= torch.Size([1, 65])
logits reshaped shape = torch.Size([1, 65])
probs tensor([[0.0091, 0.0071, 0.0053, 0.0030, 0.0141, 0.

**train more with lesser LR**
Not really helpful

In [29]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-5)  # from 1e-3

batch_size = 32  # overwrites batch_size for the generate_data function

for steps in range(2000):  # increase number of steps for good results...
    # sample a (new/random) batch of data
    xb, yb = get_batch("train")

    # print(f'input data xb = {xb}, shape = {xb.shape}')
    # print(f'target data yb = {yb}, shape = {yb.shape}')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 10 == 0:
        print(f"step: {steps} \t loss: {loss}")

print(loss.item())

step: 0 	 loss: 4.846059322357178
step: 10 	 loss: 4.682480335235596
step: 20 	 loss: 4.689340591430664
step: 30 	 loss: 4.861443996429443
step: 40 	 loss: 4.673694133758545
step: 50 	 loss: 4.7145843505859375
step: 60 	 loss: 4.83109712600708
step: 70 	 loss: 4.677046298980713
step: 80 	 loss: 4.750396728515625
step: 90 	 loss: 4.799705505371094
step: 100 	 loss: 4.705053806304932
step: 110 	 loss: 4.675622463226318
step: 120 	 loss: 4.7264404296875
step: 130 	 loss: 4.658511638641357
step: 140 	 loss: 4.747141361236572
step: 150 	 loss: 4.7842302322387695
step: 160 	 loss: 4.833553314208984
step: 170 	 loss: 4.686558723449707
step: 180 	 loss: 4.830730438232422
step: 190 	 loss: 4.706576347351074
step: 200 	 loss: 4.753474712371826
step: 210 	 loss: 4.6400275230407715
step: 220 	 loss: 4.6996235847473145
step: 230 	 loss: 4.691061019897461
step: 240 	 loss: 4.680099964141846
step: 250 	 loss: 4.6516804695129395
step: 260 	 loss: 4.873061656951904
step: 270 	 loss: 4.700374603271484
s

In [30]:
print(
    decode(
        m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[
            0
        ].tolist()
    )
)

idx initially in generate function = tensor([[0]]), shape = torch.Size([1, 1])


 max_new_tokens in loop = 0
logits shape= torch.Size([1, 1, 65]), loss = None
logits reshaped = tensor([[ 0.1965, -0.0889, -0.3785, -0.9339,  0.6067,  0.0245,  0.9353,  0.0460,
          0.3421,  1.1499, -1.3686, -0.5290,  0.2169, -0.2268, -0.9103,  1.5408,
          1.3442, -0.1378,  0.2928,  0.9648, -2.0269,  0.5032,  1.4717,  0.5908,
          0.1322, -1.5528, -1.1511, -0.3271,  0.4507, -0.7999,  1.5130,  2.4970,
         -0.6489, -0.2520,  0.9979,  0.1328,  0.1394,  1.1311, -1.1726, -0.2989,
         -0.5102, -0.9242,  0.5347, -1.4965, -1.2062,  0.5568, -0.5981, -0.6977,
          1.6262, -0.8107,  1.3330, -0.2806, -1.5100,  2.0859,  2.7437, -1.7650,
          1.4336, -1.5072,  0.8144, -0.2227,  0.7604,  1.5168,  1.5903, -0.4094,
         -0.8532]], grad_fn=<SliceBackward0>), shape= torch.Size([1, 65])
logits reshaped shape = torch.Size([1, 65])
probs tensor([[0.0093, 0.0070, 0.0052, 0.0030, 0.0140, 0.

## The mathematical trick in self-attention

In [ ]:
# help(torch.sum)

In [32]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
print(f"initial a = \n{a}")
print(f"torch.sum(a, 0) = \n{torch.sum(a, 0)}")
print(f"torch.sum(a, 1) = \n{torch.sum(a, 1)}")
print(f"torch.sum(a, 1, keepdim=True) = \n{torch.sum(a, 1, keepdim=True)}")
a = a / torch.sum(a, 1, keepdim=True)
print("a=")
print(a)


b = torch.randint(0, 10, (3, 2)).float()
print("--")
print("b=")
print(b)

c = a @ b
print("--")
print("c=")
print(c)

initial a = 
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
torch.sum(a, 0) = 
tensor([3., 2., 1.])
torch.sum(a, 1) = 
tensor([1., 2., 3.])
torch.sum(a, 1, keepdim=True) = 
tensor([[1.],
        [2.],
        [3.]])
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [33]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [34]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [40]:
# We want x[b,t] = mean_{i<=t} x[b,i]

xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]  # (t,C)
        print(f"xprev = {xprev}, shape = {xprev.shape}")
        print(f"torch.mean(xprev, 0) = {torch.mean(xprev, 0)}")
        xbow[b, t] = torch.mean(xprev, 0)

    print(f"xbow = {xbow}, shape = {xbow.shape}")

xprev = tensor([[ 0.1808, -0.0700]]), shape = torch.Size([1, 2])
torch.mean(xprev, 0) = tensor([ 0.1808, -0.0700])
xprev = tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152]]), shape = torch.Size([2, 2])
torch.mean(xprev, 0) = tensor([-0.0894, -0.4926])
xprev = tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255]]), shape = torch.Size([3, 2])
torch.mean(xprev, 0) = tensor([ 0.1490, -0.3199])
xprev = tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643]]), shape = torch.Size([4, 2])
torch.mean(xprev, 0) = tensor([ 0.3504, -0.2238])
xprev = tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679]]), shape = torch.Size([5, 2])
torch.mean(xprev, 0) = tensor([0.3525, 0.0545])
xprev = tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],

In [41]:
T

8

In [53]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
print(f"wei = {wei}")
xbow2 = wei @ x  # (B, T, T) @ (B, T, C) ----> (B, T, C)

# checkn if both results were same
print(torch.allclose(xbow, xbow2, rtol=0.01, atol=0.01))
print(f"xbow2 = {xbow2}")

wei = tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
True
xbow2 = tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         

In [54]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
print(f"tril = {tril}")
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
print(f"wei = {wei}")
wei = F.softmax(wei, dim=-1)
print(f"wei = {wei}")
xbow3 = wei @ x
torch.allclose(xbow, xbow3, rtol=0.01, atol=0.01)

tril = tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
wei = tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
wei = tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.25

True

In [55]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
wei = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)
# so here wei is not being initialised from 0 matrix or something
# it is got using the queries and keys
print(f"wei from q and k is {wei}")

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
print(f"wei from masked fill is {wei}")
wei = F.softmax(wei, dim=-1)
print(f"wei from softmax is {wei}")

v = value(x)
out = wei @ v
# out = wei @ x

print(f"out.shape is {out.shape}")

wei from q and k is tensor([[[-1.7629e+00, -1.3011e+00,  5.6516e-01,  2.1616e+00, -1.0674e+00,
           1.9632e+00,  1.0765e+00, -4.5295e-01],
         [-3.3334e+00, -1.6556e+00,  1.0405e-01,  3.3782e+00, -2.1825e+00,
           1.0415e+00, -5.5714e-02,  2.9273e-01],
         [-1.0226e+00, -1.2606e+00,  7.6228e-02, -3.8125e-01, -9.8430e-01,
          -1.4303e+00,  7.4921e-02, -9.5465e-01],
         [ 7.8359e-01, -8.0143e-01, -3.3680e-01, -8.4963e-01, -5.6023e-01,
          -1.1701e+00, -1.2927e+00, -1.0260e+00],
         [-1.2566e+00,  1.8719e-02, -7.8797e-01, -1.3204e+00,  2.0363e+00,
           8.6381e-01,  3.7188e-01,  9.2577e-01],
         [-3.1262e-01,  2.4152e+00, -1.1058e-01, -9.9305e-01,  3.3449e+00,
          -2.5229e+00,  1.4187e+00,  1.2196e+00],
         [ 1.0876e+00,  1.9652e+00, -2.6213e-01, -3.1579e-01,  6.0905e-01,
           1.2616e+00, -5.4841e-01,  8.0485e-01],
         [-1.8044e+00, -4.1260e-01, -8.3061e-01,  5.8985e-01, -7.9869e-01,
          -5.8560e-01,  6.4332

In [56]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [57]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [58]:
k.var()

tensor(1.0449)

In [59]:
q.var()

tensor(1.0700)

In [60]:
wei.var()

tensor(1.0918)

In [61]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [62]:
torch.softmax(
    torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1
)  # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [63]:
class LayerNorm1d:  # (used to be BatchNorm1d)
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True)  # batch mean
        xvar = x.var(1, keepdim=True)  # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]


torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100)  # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [66]:
x

tensor([[ 0.1335, -0.1059, -0.3824,  ..., -1.3422, -0.1971,  0.8795],
        [-0.0353, -0.7439, -0.3371,  ..., -0.6276, -0.4846,  0.4556],
        [ 0.3069, -1.5010,  1.4898,  ..., -0.6819,  0.9993,  0.8382],
        ...,
        [-1.6080, -1.6324, -0.7634,  ..., -0.9847,  0.0039, -0.8610],
        [-0.2273,  0.0066, -0.2763,  ..., -0.8705, -1.2442, -0.7531],
        [ 0.3054, -0.1505, -0.3809,  ..., -1.4962, -0.7711, -1.0681]])

In [67]:
x[:, 0]

tensor([ 0.1335, -0.0353,  0.3069,  0.6995,  0.2043,  1.7255,  1.1667,  0.3968,
        -0.9191, -0.5662,  0.4718, -2.0000, -0.1599, -1.6865,  0.6217, -0.5309,
         0.1183,  1.3908,  0.0180,  0.2807,  0.9739,  0.6511,  0.2301, -0.3239,
        -0.2884,  1.4904,  1.3951,  0.2275,  0.2371, -1.6080, -0.2273,  0.3054])

In [64]:
x[:, 0].mean(), x[:, 0].std()  # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [68]:
(
    x[0, :].mean(),
    x[0, :].std(),
)  # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [69]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

### My fully furnished code version

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5059
step 300: train loss 2.4192, val loss 2.4334
step 400: train loss 2.3499, val loss 2.3563
step 500: train loss 2.2962, val loss 2.3126
step 600: train loss 2.2411, val loss 2.2502
step 700: train loss 2.2052, val loss 2.2183
step 800: train loss 2.1639, val loss 2.1877
step 900: train loss 2.1230, val loss 2.1495
step 1000: train loss 2.1026, val loss 2.1300
step 1100: train loss 2.0683, val loss 2.1166
step 1200: train loss 2.0379, val loss 2.0796
step 1300: train loss 2.0256, val loss 2.0648
step 1400: train loss 1.9911, val loss 2.0355
step 1500: train loss 1.9694, val loss 2.0303
step 1600: train loss 1.9637, val loss 2.0484
step 1700: train loss 1.9426, val loss 2.0154
step 1800: train loss 1.9074, val loss 1.9944
step 1900: train loss 1.9105, val loss 1.9896
step 2000: train loss 1.8857, val loss 1.9949
step 2100: train loss 1.

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [70]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5059
step 300: train loss 2.4192, val loss 2.4334
step 400: train loss 2.3499, val loss 2.3563
step 500: train loss 2.2962, val loss 2.3126
step 600: train loss 2.2411, val loss 2.2502
step 700: train loss 2.2052, val loss 2.2183
step 800: train loss 2.1639, val loss 2.1877
step 900: train loss 2.1230, val loss 2.1495
step 1000: train loss 2.1026, val loss 2.1300
step 1100: train loss 2.0683, val loss 2.1166
step 1200: train loss 2.0379, val loss 2.0796
step 1300: train loss 2.0256, val loss 2.0648
step 1400: train loss 1.9911, val loss 2.0355
step 1500: train loss 1.9694, val loss 2.0303
step 1600: train loss 1.9637, val loss 2.0484
step 1700: train loss 1.9426, val loss 2.0154
step 1800: train loss 1.9074, val loss 1.9944
step 1900: train loss 1.9105, val loss 1.9896
step 2000: train loss 1.8857, val loss 1.9949
step 2100: train loss 1.